In [1]:

import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('DEBUG')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)

import os
os.sys.path.append('/home/schirrmr/braindecode/code/fbcsp/')
os.sys.path.append('/home/schirrmr/braindecode/code/braindecode/')

%load_ext autoreload
%autoreload 2
%cd /home/schirrmr/

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 7

import logging
log = logging.getLogger()
log.setLevel('DEBUG')

/home/schirrmr


In [18]:
from braindecode.datasets.bbci import BBCIDataset
set_loader = BBCIDataset('data/BBCI-only-last-runs/BhNoMoSc1S001R13_ds10_1-2BBCI.mat',
                        load_sensor_names=['C3', 'C4', 'C1', 'C2', 'CPz', 'F3', 'F4', 'Fz'])
cnt = set_loader.load()
cnt = cnt.drop_channels(['STI 014'])
eog_cnt = BBCIDataset('data/BBCI-only-last-runs/BhNoMoSc1S001R13_ds10_1-2BBCI.mat',
                        load_sensor_names=['EOGh', 'EOGv']).load()
eog_cnt = eog_cnt.drop_channels(['STI 014'])

Creating RawArray with float64 data, n_channels=8, n_times=617090
    Range : 0 ... 617089 =      0.000 ...  1234.178 secs
Ready.
Creating RawArray with float64 data, n_channels=2, n_times=617090
    Range : 0 ... 617089 =      0.000 ...  1234.178 secs
Ready.


In [19]:
from braindecode.mne_ext.signalproc import mne_apply
from braindecode.datautil.signalproc import bandpass_cnt

import numpy as np
def car_cnt(cnt):
    return mne_apply(lambda x: x - np.mean(x,axis=0, keepdims=True), cnt)

cnt = car_cnt(cnt)
cnt = mne_apply(lambda x: bandpass_cnt(x, 0.5,150.0, cnt.info['sfreq']), cnt)

In [20]:
from fbcsp.experiment import CSPExperiment
from collections import OrderedDict
name_to_start_codes = OrderedDict([('Right', 1), ('Left', 2), ('Rest',3)])
epoch_ival_ms = [0,4000]

In [21]:
from fbcsp.clean import EOGMaxMinCleaner, VarCleaner, apply_multiple_cleaners


cleaners = [EOGMaxMinCleaner(eog_cnt, epoch_ival_ms, name_to_start_codes, threshold=600),
           VarCleaner(whisker_percent=10, whisker_length=3)]
cleaned_cnt = apply_multiple_cleaners(cnt, epoch_ival_ms, name_to_start_codes, cleaners)

2017-09-21 15:51:18,222 INFO : Apply EOGMaxMinCleaner...
2017-09-21 15:51:18,223 INFO : Cleaning...
2017-09-21 15:51:18,247 INFO : Trial per class:
Counter({'Left': 40, 'Rest': 40, 'Right': 40})
2017-09-21 15:51:18,257 INFO : Trial per class:
Counter({'Left': 40, 'Rest': 40, 'Right': 40})
2017-09-21 15:51:18,259 INFO : Rejected channels: []
2017-09-21 15:51:18,259 INFO : #Clean trials:     112
2017-09-21 15:51:18,260 INFO : #Rejected trials:  8
2017-09-21 15:51:18,261 INFO : Fraction Clean:    93.3%
2017-09-21 15:51:18,314 INFO : Apply VarCleaner...
2017-09-21 15:51:18,315 INFO : Cleaning...
2017-09-21 15:51:18,336 INFO : Trial per class:
Counter({'Left': 38, 'Right': 38, 'Rest': 36})
2017-09-21 15:51:18,344 INFO : Rejected channels: []
2017-09-21 15:51:18,345 INFO : #Clean trials:     112
2017-09-21 15:51:18,346 INFO : #Rejected trials:  0
2017-09-21 15:51:18,346 INFO : Fraction Clean:    100.0%


In [28]:
exp = CSPExperiment(cleaned_cnt, name_to_start_codes, epoch_ival_ms,
                    min_freq = 0,
                    max_freq=144, last_low_freq=30,
                    low_width=2,
                    high_width=6,
                    low_overlap=0,
                    high_overlap=0,
                    filt_order=3,
                    n_folds=10,
                    n_top_bottom_csp_filters=5,
                    n_selected_filterbands=None,
                    n_selected_features=20,
                    forward_steps=2,
                    backward_steps=1,)

In [29]:
from braindecode.datautil.trial_segment import create_signal_target, create_signal_target_from_raw_mne

from braindecode.datautil.signalproc import bandpass_cnt

from braindecode.datautil.iterators import get_balanced_batches

In [30]:
exp.run()

2017-09-21 15:58:52,838 INFO : Trial per class:
Counter({'Left': 38, 'Right': 38, 'Rest': 36})
2017-09-21 15:58:52,846 INFO : Running Training...
2017-09-21 15:58:52,848 INFO : Filter 1/35, 0.20 to 1.00 Hz
2017-09-21 15:58:52,987 INFO : Trial per class:
Counter({'Left': 38, 'Right': 38, 'Rest': 36})
2017-09-21 15:58:52,990 INFO : Fold Nr: 1
2017-09-21 15:58:52,994 INFO : Class 1 vs 2
2017-09-21 15:58:53,100 INFO : Train: 80.88%
2017-09-21 15:58:53,101 INFO : Test:  37.50%
2017-09-21 15:58:53,103 INFO : Class 1 vs 3
2017-09-21 15:58:53,204 INFO : Train: 81.82%
2017-09-21 15:58:53,205 INFO : Test:  62.50%
2017-09-21 15:58:53,207 INFO : Class 2 vs 3
2017-09-21 15:58:53,307 INFO : Train: 86.36%
2017-09-21 15:58:53,309 INFO : Test:  75.00%
2017-09-21 15:58:53,310 INFO : Fold Nr: 2
2017-09-21 15:58:53,312 INFO : Class 1 vs 2
2017-09-21 15:58:53,378 INFO : Train: 86.76%
2017-09-21 15:58:53,379 INFO : Test:  50.00%
2017-09-21 15:58:53,380 INFO : Class 1 vs 3
2017-09-21 15:58:53,444 INFO : Trai

2017-09-21 15:58:57,071 INFO : Train: 85.29%
2017-09-21 15:58:57,072 INFO : Test:  50.00%
2017-09-21 15:58:57,073 INFO : Class 2 vs 3
2017-09-21 15:58:57,141 INFO : Train: 83.58%
2017-09-21 15:58:57,143 INFO : Test:  85.71%
2017-09-21 15:58:57,144 INFO : Fold Nr: 9
2017-09-21 15:58:57,146 INFO : Class 1 vs 2
2017-09-21 15:58:57,214 INFO : Train: 85.51%
2017-09-21 15:58:57,215 INFO : Test:  28.57%
2017-09-21 15:58:57,216 INFO : Class 1 vs 3
2017-09-21 15:58:57,291 INFO : Train: 90.91%
2017-09-21 15:58:57,292 INFO : Test:  50.00%
2017-09-21 15:58:57,293 INFO : Class 2 vs 3
2017-09-21 15:58:57,363 INFO : Train: 83.58%
2017-09-21 15:58:57,364 INFO : Test:  85.71%
2017-09-21 15:58:57,365 INFO : Fold Nr: 10
2017-09-21 15:58:57,368 INFO : Class 1 vs 2
2017-09-21 15:58:57,437 INFO : Train: 82.35%
2017-09-21 15:58:57,438 INFO : Test:  87.50%
2017-09-21 15:58:57,439 INFO : Class 1 vs 3
2017-09-21 15:58:57,505 INFO : Train: 89.55%
2017-09-21 15:58:57,506 INFO : Test:  71.43%
2017-09-21 15:58:57,5

2017-09-21 15:59:01,027 INFO : Test:  62.50%
2017-09-21 15:59:01,028 INFO : Class 1 vs 3
2017-09-21 15:59:01,095 INFO : Train: 89.39%
2017-09-21 15:59:01,096 INFO : Test:  75.00%
2017-09-21 15:59:01,097 INFO : Class 2 vs 3
2017-09-21 15:59:01,163 INFO : Train: 82.35%
2017-09-21 15:59:01,164 INFO : Test:  100.00%
2017-09-21 15:59:01,165 INFO : Fold Nr: 7
2017-09-21 15:59:01,167 INFO : Class 1 vs 2
2017-09-21 15:59:01,233 INFO : Train: 86.96%
2017-09-21 15:59:01,234 INFO : Test:  57.14%
2017-09-21 15:59:01,235 INFO : Class 1 vs 3
2017-09-21 15:59:01,300 INFO : Train: 88.06%
2017-09-21 15:59:01,301 INFO : Test:  71.43%
2017-09-21 15:59:01,302 INFO : Class 2 vs 3
2017-09-21 15:59:01,369 INFO : Train: 83.33%
2017-09-21 15:59:01,370 INFO : Test:  62.50%
2017-09-21 15:59:01,371 INFO : Fold Nr: 8
2017-09-21 15:59:01,373 INFO : Class 1 vs 2
2017-09-21 15:59:01,438 INFO : Train: 83.58%
2017-09-21 15:59:01,439 INFO : Test:  77.78%
2017-09-21 15:59:01,440 INFO : Class 1 vs 3
2017-09-21 15:59:01,50

2017-09-21 15:59:04,976 INFO : Class 1 vs 2
2017-09-21 15:59:05,045 INFO : Train: 80.00%
2017-09-21 15:59:05,046 INFO : Test:  50.00%
2017-09-21 15:59:05,047 INFO : Class 1 vs 3
2017-09-21 15:59:05,115 INFO : Train: 93.94%
2017-09-21 15:59:05,116 INFO : Test:  62.50%
2017-09-21 15:59:05,117 INFO : Class 2 vs 3
2017-09-21 15:59:05,181 INFO : Train: 95.45%
2017-09-21 15:59:05,182 INFO : Test:  75.00%
2017-09-21 15:59:05,183 INFO : Fold Nr: 5
2017-09-21 15:59:05,186 INFO : Class 1 vs 2
2017-09-21 15:59:05,251 INFO : Train: 76.81%
2017-09-21 15:59:05,252 INFO : Test:  57.14%
2017-09-21 15:59:05,253 INFO : Class 1 vs 3
2017-09-21 15:59:05,320 INFO : Train: 92.54%
2017-09-21 15:59:05,321 INFO : Test:  100.00%
2017-09-21 15:59:05,322 INFO : Class 2 vs 3
2017-09-21 15:59:05,389 INFO : Train: 90.91%
2017-09-21 15:59:05,390 INFO : Test:  100.00%
2017-09-21 15:59:05,391 INFO : Fold Nr: 6
2017-09-21 15:59:05,394 INFO : Class 1 vs 2
2017-09-21 15:59:05,461 INFO : Train: 79.41%
2017-09-21 15:59:05,4

/home/schirrmr/braindecode/code/fbcsp/fbcsp/signalproc.py:229: ComplexWarning: Casting complex values to real discards the imaginary part
  filtered[trial_i] = np.dot(epo.X[trial_i].T, f)


...

2017-09-21 15:59:57,771 INFO : Test:  25.00%
2017-09-21 15:59:57,772 INFO : Class 1 vs 3
2017-09-21 15:59:57,839 INFO : Train: 73.13%
2017-09-21 15:59:57,840 INFO : Test:  14.29%
2017-09-21 15:59:57,842 INFO : Class 2 vs 3
2017-09-21 15:59:57,907 INFO : Train: 68.66%
2017-09-21 15:59:57,908 INFO : Test:  42.86%
2017-09-21 15:59:57,909 INFO : Filter 30/35, 111.00 to 117.00 Hz
2017-09-21 15:59:58,029 INFO : Trial per class:
Counter({'Left': 38, 'Right': 38, 'Rest': 36})
2017-09-21 15:59:58,031 INFO : Fold Nr: 1
2017-09-21 15:59:58,033 INFO : Class 1 vs 2
2017-09-21 15:59:58,098 INFO : Train: 73.53%
2017-09-21 15:59:58,099 INFO : Test:  75.00%
2017-09-21 15:59:58,100 INFO : Class 1 vs 3
2017-09-21 15:59:58,165 INFO : Train: 75.76%
2017-09-21 15:59:58,166 INFO : Test:  25.00%
2017-09-21 15:59:58,167 INFO : Class 2 vs 3
2017-09-21 15:59:58,237 INFO : Train: 69.70%
2017-09-21 15:59:58,238 INFO : Test:  50.00%
2017-09-21 15:59:58,239 INFO : Fold Nr: 2
2017-09-21 15:59:58,241 INFO : Class 1 vs

2017-09-21 16:00:01,777 INFO : Class 1 vs 2
2017-09-21 16:00:01,841 INFO : Train: 77.61%
2017-09-21 16:00:01,842 INFO : Test:  66.67%
2017-09-21 16:00:01,843 INFO : Class 1 vs 3
2017-09-21 16:00:01,911 INFO : Train: 73.53%
2017-09-21 16:00:01,912 INFO : Test:  66.67%
2017-09-21 16:00:01,913 INFO : Class 2 vs 3
2017-09-21 16:00:01,979 INFO : Train: 74.63%
2017-09-21 16:00:01,980 INFO : Test:  28.57%
2017-09-21 16:00:01,982 INFO : Fold Nr: 9
2017-09-21 16:00:01,984 INFO : Class 1 vs 2
2017-09-21 16:00:02,050 INFO : Train: 69.57%
2017-09-21 16:00:02,051 INFO : Test:  57.14%
2017-09-21 16:00:02,052 INFO : Class 1 vs 3
2017-09-21 16:00:02,116 INFO : Train: 65.15%
2017-09-21 16:00:02,118 INFO : Test:  62.50%
2017-09-21 16:00:02,119 INFO : Class 2 vs 3
2017-09-21 16:00:02,185 INFO : Train: 61.19%
2017-09-21 16:00:02,186 INFO : Test:  57.14%
2017-09-21 16:00:02,187 INFO : Fold Nr: 10
2017-09-21 16:00:02,189 INFO : Class 1 vs 2
2017-09-21 16:00:02,254 INFO : Train: 73.53%
2017-09-21 16:00:02,25

2017-09-21 16:00:06,033 INFO : Test:  25.00%
2017-09-21 16:00:06,034 INFO : Fold Nr: 6
2017-09-21 16:00:06,037 INFO : Class 1 vs 2
2017-09-21 16:00:06,130 INFO : Train: 72.06%
2017-09-21 16:00:06,131 INFO : Test:  37.50%
2017-09-21 16:00:06,132 INFO : Class 1 vs 3
2017-09-21 16:00:06,222 INFO : Train: 71.21%
2017-09-21 16:00:06,223 INFO : Test:  50.00%
2017-09-21 16:00:06,224 INFO : Class 2 vs 3
2017-09-21 16:00:06,314 INFO : Train: 72.06%
2017-09-21 16:00:06,316 INFO : Test:  50.00%
2017-09-21 16:00:06,317 INFO : Fold Nr: 7
2017-09-21 16:00:06,319 INFO : Class 1 vs 2
2017-09-21 16:00:06,406 INFO : Train: 71.01%
2017-09-21 16:00:06,408 INFO : Test:  71.43%
2017-09-21 16:00:06,409 INFO : Class 1 vs 3
2017-09-21 16:00:06,495 INFO : Train: 70.15%
2017-09-21 16:00:06,497 INFO : Test:  42.86%
2017-09-21 16:00:06,500 INFO : Class 2 vs 3
2017-09-21 16:00:06,588 INFO : Train: 74.24%
2017-09-21 16:00:06,589 INFO : Test:  50.00%
2017-09-21 16:00:06,590 INFO : Fold Nr: 8
2017-09-21 16:00:06,592 I

2017-09-21 16:00:10,073 INFO : Class 2 vs 3
2017-09-21 16:00:10,138 INFO : Train: 64.18%
2017-09-21 16:00:10,139 INFO : Test:  57.14%
2017-09-21 16:00:10,140 INFO : Fold Nr: 4
2017-09-21 16:00:10,142 INFO : Class 1 vs 2
2017-09-21 16:00:10,207 INFO : Train: 67.14%
2017-09-21 16:00:10,208 INFO : Test:  33.33%
2017-09-21 16:00:10,209 INFO : Class 1 vs 3
2017-09-21 16:00:10,275 INFO : Train: 75.76%
2017-09-21 16:00:10,276 INFO : Test:  62.50%
2017-09-21 16:00:10,277 INFO : Class 2 vs 3
2017-09-21 16:00:10,344 INFO : Train: 71.21%
2017-09-21 16:00:10,345 INFO : Test:  62.50%
2017-09-21 16:00:10,346 INFO : Fold Nr: 5
2017-09-21 16:00:10,349 INFO : Class 1 vs 2
2017-09-21 16:00:10,414 INFO : Train: 65.22%
2017-09-21 16:00:10,415 INFO : Test:  14.29%
2017-09-21 16:00:10,416 INFO : Class 1 vs 3
2017-09-21 16:00:10,482 INFO : Train: 73.13%
2017-09-21 16:00:10,483 INFO : Test:  28.57%
2017-09-21 16:00:10,484 INFO : Class 2 vs 3
2017-09-21 16:00:10,560 INFO : Train: 72.73%
2017-09-21 16:00:10,561

2017-09-21 16:02:43,640 INFO : Train: 100.00%
2017-09-21 16:02:43,641 INFO : Test:  81.82%
2017-09-21 16:02:43,642 INFO : Fold Nr: 4
2017-09-21 16:02:43,644 INFO : Train: 99.01%
2017-09-21 16:02:43,644 INFO : Test:  63.64%
2017-09-21 16:02:43,645 INFO : Fold Nr: 5
2017-09-21 16:02:43,647 INFO : Train: 99.01%
2017-09-21 16:02:43,648 INFO : Test:  90.91%
2017-09-21 16:02:43,648 INFO : Fold Nr: 6
2017-09-21 16:02:43,650 INFO : Train: 100.00%
2017-09-21 16:02:43,651 INFO : Test:  81.82%
2017-09-21 16:02:43,652 INFO : Fold Nr: 7
2017-09-21 16:02:43,653 INFO : Train: 99.01%
2017-09-21 16:02:43,654 INFO : Test:  90.91%
2017-09-21 16:02:43,655 INFO : Fold Nr: 8
2017-09-21 16:02:43,656 INFO : Train: 100.00%
2017-09-21 16:02:43,657 INFO : Test:  100.00%
2017-09-21 16:02:43,658 INFO : Fold Nr: 9
2017-09-21 16:02:43,659 INFO : Train: 100.00%
2017-09-21 16:02:43,660 INFO : Test:  100.00%
2017-09-21 16:02:43,661 INFO : Fold Nr: 10
2017-09-21 16:02:43,662 INFO : Train: 100.00%
2017-09-21 16:02:43,663